In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!ls

base  data_repository  notebooks  requirements.txt  src


In [23]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


In [24]:
!pip install torch_geometric

In [25]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


# Requirements

In [26]:
import torch

from torch.optim import Adam
from torch_geometric.nn import ComplEx, DistMult, RotatE, TransE
from torch_geometric.data import Data

from base import OptimizerConfig, cross_validation
from base import SimplePytorchData, SimplePytorchDataTrainTestSplit
from base import SimpleTrainer, SimpleTester
from src.config import SimpleClassifierConfig, GraphAutoEncoderConfig, KGEConfig
from src.features import get_relations, get_entities, get_associations, get_homogeneous_graph, get_kge_pair_embedd_for_training_data
from src.models import SimpleMDAClassifier, SimpleMDAClassifierFactory
from src.utils import train_test_sampler, prj_logger
from torch_geometric.nn import GCNConv

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [28]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

# RotatE

## Config

In [29]:
kge_optimizer_config = OptimizerConfig()
kge_optimizer_config.optimizer = torch.optim.Adam
kge_optimizer_config.lr = 0.01
kge_optimizer_config.batch_size = 1000
kge_optimizer_config.n_epoch = 10
kge_optimizer_config.exp_name = "Optimizer for Graph Auto Encoder"
kge_optimizer_config.device = device
kge_optimizer_config.report_size = device

In [30]:
kge_model_config = KGEConfig()
kge_model_config.kge = RotatE
kge_model_config.hidden_channels = 32

## Embedding

In [31]:
md_embed = get_kge_pair_embedd_for_training_data(kge_model_config, kge_optimizer_config)

2024-02-12 10:06:25,616 [INFO] Calling get_node2vec_pair_embedd on cpu device ...
2024-02-12 10:06:25,621 [INFO] Calling get_homogeneous_graph
2024-02-12 10:06:27,402 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])
2024-02-12 10:06:27,410 [INFO] Calling get_kge_embedd on cpu device ...
2024-02-12 10:06:27,413 [INFO] Calling get_knowledge_graph
2024-02-12 10:06:28,568 [INFO] knowledge graph data : Data(num_nodes=66911, edge_index=[2, 633662], edge_type=[633662])
2024-02-12 10:06:28,573 [INFO] Setting num relations and num nodes for kge config to 39 and 66911
2024-02-12 10:06:28,577 [INFO] Creating KGE model ...
2024-02-12 10:06:28,579 [INFO] Initialing MDATransE with model_config {'model_name': None}
2024-02-12 10:06:28,647 [INFO] Training KGE ...
2024-02-12 10:06:28,649 [INFO] Running KGETrainer with Optimizer for Graph Auto Encoder
2024-02-12 10:06:28,651 [INFO] Creating <class 'torch.optim.adam.Adam'> with lr : 0.01
2024-02-12 10:06:28,654 [INFO] moving model to 

# Classification

## Data

In [32]:
associations = get_associations()
y = torch.tensor(associations['increased'].tolist(), dtype=torch.float32).reshape(-1, 1).to(device)

In [33]:
# Train Test Split
train_indices, test_indices = train_test_sampler(y.shape[0], 0.7)

data = SimplePytorchData(md_embed, y)
train_data = SimplePytorchData(md_embed[train_indices], y[train_indices])
test_data = SimplePytorchData(md_embed[test_indices], y[test_indices])

2024-02-12 10:12:52,144 [INFO] Initializing SimplePytorchData with X shape : torch.Size([898, 64]) and y shape : torch.Size([898, 1])
2024-02-12 10:12:52,150 [INFO] Initializing SimplePytorchData with X shape : torch.Size([628, 64]) and y shape : torch.Size([628, 1])
2024-02-12 10:12:52,152 [INFO] Initializing SimplePytorchData with X shape : torch.Size([270, 64]) and y shape : torch.Size([270, 1])


## Classifier

In [34]:
simple_classifier_config = SimpleClassifierConfig()
simple_classifier_config.model_name = "simple classifier"
simple_classifier_config.input_dim = md_embed.shape[1]
simple_classifier_config.hidden_dim = 32
simple_classifier_config.output_dim = 1
simple_classifier_config.num_layers = 2
simple_classifier_config.dropout = 0.1

In [35]:
mda_classifier = SimpleMDAClassifier(simple_classifier_config)

2024-02-12 10:12:52,172 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-12 10:12:52,176 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout


## Optimizer

In [36]:
classifier_optimizer_config = OptimizerConfig()
classifier_optimizer_config.optimizer = torch.optim.Adam
classifier_optimizer_config.criterion = torch.nn.BCEWithLogitsLoss()
classifier_optimizer_config.lr = 0.01
classifier_optimizer_config.batch_size = 32
classifier_optimizer_config.n_epoch = 50
classifier_optimizer_config.exp_name = "adam optimizer"
classifier_optimizer_config.save = False
classifier_optimizer_config.save_path = None
classifier_optimizer_config.device = device
classifier_optimizer_config.report_size = 10  # batch to report ratio
classifier_optimizer_config.threshold = 0.5

## Train Test Approach

### Train

In [37]:
train_result = SimpleTrainer().train(model=mda_classifier,
                                     data=train_data,
                                     config=classifier_optimizer_config)

2024-02-12 10:12:52,196 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-12 10:12:52,198 [INFO] moving data and model to cpu
2024-02-12 10:12:52,228 [INFO] loss: 0.0212    [1,    10]
2024-02-12 10:12:52,247 [INFO] loss: 0.0198    [1,    20]
2024-02-12 10:12:52,267 [INFO] loss: 0.0199    [2,    10]
2024-02-12 10:12:52,289 [INFO] loss: 0.0189    [2,    20]
2024-02-12 10:12:52,309 [INFO] loss: 0.0199    [3,    10]
2024-02-12 10:12:52,328 [INFO] loss: 0.0182    [3,    20]
2024-02-12 10:12:52,348 [INFO] loss: 0.0182    [4,    10]
2024-02-12 10:12:52,367 [INFO] loss: 0.0200    [4,    20]
2024-02-12 10:12:52,387 [INFO] loss: 0.0190    [5,    10]
2024-02-12 10:12:52,408 [INFO] loss: 0.0186    [5,    20]
2024-02-12 10:12:52,427 [INFO] loss: 0.0187    [6,    10]
2024-02-12 10:12:52,446 [INFO] loss: 0.0186    [6,    20]
2024-02-12 10:12:52,464 [INFO] loss: 0.0188    [7,    10]
2024-02-12 10:12:52,482 [INFO] loss: 0.0182    [7,    20]
2024-02-12 10:12:52,500 [INFO] loss: 0.0184  

### Test

In [38]:
test_result = SimpleTester().test(model=mda_classifier,
                                  data=test_data,
                                  config=classifier_optimizer_config)

2024-02-12 10:12:54,514 [INFO] Running Simple Tester with config : adam optimizer
2024-02-12 10:12:54,517 [INFO] moving data and model to cpu
2024-02-12 10:12:54,548 [INFO] Result on Test Data : {'AUC': 0.8575034293552812, 'ACC': 0.7222222222222222, 'F1 Score': 0.7119446933811291, 'AUPR': 0, 'Loss': 0.5303164654307895}


In [39]:
test_result.get_result()

{'AUC': 0.8575034293552812,
 'ACC': 0.7222222222222222,
 'F1 Score': 0.7119446933811291,
 'AUPR': 0,
 'Loss': 0.5303164654307895}

## Cross Validation

In [40]:
trainer = SimpleTrainer()
tester = SimpleTester()
factory = SimpleMDAClassifierFactory(simple_classifier_config)
spliter = SimplePytorchDataTrainTestSplit(data)
cross_validation(k=5, data_size=data.X.shape[0], train_test_spliter=spliter, model_factory=factory,
                    trainer=trainer, tester=tester, config=classifier_optimizer_config)

2024-02-12 10:12:54,572 [INFO] Initializing SimpleMDAClassifierFactory with model : simple classifier
2024-02-12 10:12:54,574 [INFO] Initializing SimplePytorchDataTrainTestSplit
2024-02-12 10:12:54,578 [INFO] Start 5-fold Cross Validation with config : adam optimizer
2024-02-12 10:12:54,580 [INFO] ---- Fold 1 ----
2024-02-12 10:12:54,583 [INFO] Initializing SimplePytorchData with X shape : torch.Size([719, 64]) and y shape : torch.Size([719, 1])
2024-02-12 10:12:54,586 [INFO] Initializing SimplePytorchData with X shape : torch.Size([179, 64]) and y shape : torch.Size([179, 1])
2024-02-12 10:12:54,587 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-12 10:12:54,589 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout
2024-02-12 10:12:54,591 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-12 10:12:54,593 [INFO] moving data and model to cpu
2024-02-12 10:12:54,